In [11]:
def crear_datos():
  #Descargar archivo  
  #Open, readlines, close
  r = open("data/original/oneline.txt")
  ls = r.readlines()
  r.close()

  #Leer numero de numeros y medidas
  n_nums = int(ls.pop(0).strip())
  n_fils = int(ls.pop(0).strip())
  n_cols = int(ls.pop(0).strip())

  DAT = []
  RES = []
  #Procesar numero a numero
  for n in range(0,n_nums):
    #Numero correcto en string
    sres = ls.pop(0).strip()
    #Dibujo del numero en string
    sdat = ls.pop(0).strip()

    res = convertir_res(sres)
    RES.append(res)
    dat = convertir_dat(sdat)
    DAT.append(dat)

  return DAT,RES

def convertir_res(sres):
  res = []
  for z in range(0,N_Z):
    res.append(0.0)
  res[int(sres)] = 1.0
  return res

def convertir_dat(sdat):
  dat = []
  for c in sdat:
    dat.append(int(c))
  return dat


In [12]:
def crear_red():
  x = []          # primera capa
  for _ in range(0,N_X):
    x.append(0)

  w1 = []       #segunda capa
  for y in range(0,N_Y):
    W = []
    for x in range(0,N_X):
      r = (random.randint(0,100)/100)-0.5
      W.append(r)
    w1.append(W)


  #b1 #primer bias
  b1 = []
  for y in range(0, N_Y):
    b1.append((random.randint(0,100)/100)-0.5)

  #b2      #segundo bias 
  b2 = []
  for z in range(0, N_Z):
    b2.append((random.randint(0,100)/100)-0.5)




  w2 = []       # pesos tercera capa
  for z in range(0,N_Z):
    W = []
    for x in range(0,N_Y):
      r = (random.randint(0,100)/100)-0.5
      W.append(r)
    w2.append(W)



  return x, w1, w2,b1, b2

# New Section

In [13]:
def get_loss(A2,y):
  return -math.log(A2[y.index(max(y))])

In [1]:
def entrenar(nums,x,  w1, w2,b1, b2, start):
  
  #Para cada numero de mis datos
  END = 20000
  for i in range(nums):
    corrects = 0
    loss = 0
    for n in range(0,END):
      x = DAT[n]
      y = RES[n]

      Z1, A1, Z2, A2  = forward(x, w1, b1, w2, b2)      # generamos el output
      dW1, db1, dW2, db2 = back(Z1, A1, A2,  w2, x, y)  # back propagation
      w1, b1, w2, b2 = actualizar(w1,b1, w2,b2, dW1, dW2,db1, db2, LRATE,END)   #update weights
      if i%10 == 0:
        if A2.index(max(A2)) == y.index(max(y)):
          corrects +=1
        loss += get_loss(A2,y)

    if i %10 == 0:
      stop = timeit.default_timer()
      print("tiempo entre rondas:","{:.2f}".format(stop-start),'segundos' )
      start = stop
      print("Round",i)
      print("average hasta ahora de:" ,corrects/END)
      print(f"loss: {loss/END}")
      corrects = 0
  return w1, b1, w2, b2

In [15]:
def forward(x, w1, b1, w2, b2):    #FORWARD PROPAGATION
  Z1, A1 = forwardX2Y(x,w1,b1)
  Z2, A2 = forwardY2Z(w2,A1,b2)
  return Z1, A1, Z2, A2

def forwardX2Y(x,w1,b1):
  Z1 = []
  A1 = []
  for y in range(0,N_Y):
    z1, a1 =  forwardX2Yone(x,y,w1,b1)
    Z1.append(z1)
    A1.append(a1)
  return Z1, A1   #A1 es el output de la segunda capa

def forwardX2Yone(x,y,w1,b1):
  activation = 0
  for i in range(0,N_X):
    activation += x[i]*w1[y][i]

  activation += b1[y]
  return activation, max(activation, 0)


def forwardY2Z(w2,A1,b2):
  Z2 = []
  A2 = []
  for z in range(0,N_Z):
    z2 = forwardY2Zone(z,w2,A1,b2)
    Z2.append(z2)
  A2 = softmax(Z2, A2)
  return Z2, A2 #A2 es el output de la tercera capa
  
def forwardY2Zone(z,w2,A1, b2):
  activation = 0
  
  for i in range(0,N_Y):
    activation += A1[i]*w2[z][i]
  activation += b2[z]   

  return activation


def softmax(Z2,A2):
  A2 = []
  general = 0
  for z in range(0, N_Z):
    general += math.exp(Z2[z])
  for z in range(0, N_Z):
    A2.append(math.exp(Z2[z])/general)
  return A2

In [16]:
def mult_matrix(a,b):
    matrix = []
    for i in range(len(a)):
      matrix.append([])
      for j in range(len(b[0])):
          matrix[i].append(0)

    for i in range(len(a)):
      for j in range(len(b[0])):
        for k in range(len(a[0])):
          matrix[i][j]  += a[i][k] * b[k][j]
    return matrix

def transpose(matrix):

    result = []
    for i in range(len(matrix[0])):
      partial = []
      for j in range(len(matrix)):
        partial.append(0)
      result.append(partial)

    for i in range(len(matrix[0])):
        for j in range(len(matrix)):
            result[i][j] = matrix[j][i] 
    return result

In [17]:

def back(Z1, A1, A2,  w2, x, y):   #back propagation ERROR
  dZ2, dW2 = backZ(A1, A2, y)
  w2T = transpose(w2)
  dW1, dZ1 = backY(x, w2T, Z1, dZ2)
  db2 = 0
  for value in dZ2:
     db2 +=value[0]
  db1 = 0
  for value in dZ1:
     db1 +=value[0]
  return dW1, db1, dW2, db2
  

def backZ( A1,  A2,  y):
  dZ2 = []
  for z in range(0,N_Z):    #ACTUALIZAMOS Z CON LOS OUTPUTS Y SU ERROR
    #Error
    Error = [A2[z] - y[z]]
    dZ2.append(Error)

  definiti = []
  definiti.append(A1)
  dW2 = mult_matrix(dZ2, definiti)
  return dZ2, dW2

def backY(x, w2T, Z1, dZ2):
  dZ1 = mult_matrix(w2T, dZ2)
  for y in range(0,N_Y):  
    if Z1[y]>0:
      dZ1[y] = dZ1[y]
    else:
      dZ1[y] = [0]
  xT = [x]
  dW1 = mult_matrix(dZ1, xT)
  return dW1, dZ1
  



In [18]:
def actualizar(w1,b1, w2,b2, dW1, dW2, db1, db2, LRATE, END):
  w1 = actualizarY(w1, dW1, LRATE, END)
  b1 = actualizarB1(b1,db1, LRATE, END)
  w2 = actualizarZ(w2, dW2, LRATE, END)
  b2 = actualizarB2(b2,db2, LRATE, END)
  return w1, b1,w2,b2

def actualizarB1(b1, db1, LRATE, END):
  for i in range(len(b1)):
    b1[i] = b1[i] - LRATE*db1/END #dividimos por END para que sea el ponderado de todos los casos en una pasada
  return b1

def actualizarB2(b2, db2, LRATE, END):
  for i in range(len(b2)):
    b2[i] = b2[i] - LRATE*db2/END #dividimos por END para que sea el ponderado de todos los casos en una pasada
  return b2

def actualizarY(w1, dW1, LRATE, END):
  for y in range(0,N_Y):
    for x in range(0,N_X):
      w1[y][x] -= LRATE *dW1[y][x]/END #dividimos por END para que sea el ponderado de todos los casos en una pasada
  return w1

def actualizarZ(w2, dW2, LRATE, END):
  for z in range(0,N_Z):
    for y in range(0,N_Y):
      w2[z][y] -= LRATE *dW2[z][y]/END #dividimos por END para que sea el ponderado de todos los casos en una pasada
  return w2


In [19]:
def predecir(X, w1,b1, w2, b2):
  _, _, _, A2 = forward(X,w1, b1, w2,  b2)
  #Buscar que Z es el numero maximo
  maxpos = -1
  maxval = -1.0
  for z in range(0,N_Z):
    if A2[z] > maxval:
      maxval = A2[z]
      maxpos = z
  return maxpos

In [20]:
#CODIGO PRINCIPAL (Part 1)
import random
import math

#Configuracion
N_X = 28*28
N_Y = 40
N_Z = 10 
LRATE = 0.1


#Crear red y datos  !wget -O oneline.txt https://www.dropbox.com/s/ps958u765f2jabe/oneline.txt?dl=0

DAT,RES = crear_datos()

In [30]:
import timeit
#CODIGO PRINCIPAL (Part 2)

start = timeit.default_timer()
#Entrenarx
X,  w1, w2, b1, b2 = crear_red()        
w1, b1,w2,b2 = entrenar(500, X,  w1, w2, b1, b2, start) #probamos con 400 imagenes


#Predecir
X = convertir_dat(
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000001100000000000000000"+
"0000000001100000000000000000"+
"0000000011100000011100000000"+
"0000000011100000011100000000"+
"0000000011100000111000000000"+
"0000000011000000111000000000"+
"0000000111000000111000000000"+
"0000000111000000111000000000"+
"0000000111000001111000000000"+
"0000000111000001111111100000"+
"0000000111111111111111100000"+
"0000000111111111111111000000"+
"0000000001111111110000000000"+
"0000000000000011100000000000"+
"0000000000000011100000000000"+
"0000000000000011100000000000"+
"0000000000000011100000000000"+
"0000000000000111100000000000"+
"0000000000000111100000000000"+
"0000000000000111100000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000"+
"0000000000000000000000000000")
num = predecir(X,w1,b1,w2,b2)
print ("El numero es un", num, "y era un ", 4)


tiempo entre rondas: 439.97 segundos
Round 0
average hasta ahora de: 0.1336
tiempo entre rondas: 4348.82 segundos
Round 10
average hasta ahora de: 0.43225
tiempo entre rondas: 4273.23 segundos
Round 20
average hasta ahora de: 0.57525
tiempo entre rondas: 4267.38 segundos
Round 30
average hasta ahora de: 0.6481
tiempo entre rondas: 4171.38 segundos
Round 40
average hasta ahora de: 0.6902
tiempo entre rondas: 4986.70 segundos
Round 50
average hasta ahora de: 0.71795
tiempo entre rondas: 5681.03 segundos
Round 60
average hasta ahora de: 0.73815
tiempo entre rondas: 5630.22 segundos
Round 70
average hasta ahora de: 0.75515
tiempo entre rondas: 5689.80 segundos
Round 80
average hasta ahora de: 0.76735
tiempo entre rondas: 5557.46 segundos
Round 90
average hasta ahora de: 0.77895
tiempo entre rondas: 4151.32 segundos
Round 100
average hasta ahora de: 0.78785
tiempo entre rondas: 4181.31 segundos
Round 110
average hasta ahora de: 0.7949
tiempo entre rondas: 5501.70 segundos
Round 120
average 

In [92]:
X_test = DAT[42000:]
y_test = RES[42000:]
number = 0
for i in range(len(y_test)):  
  X = X_test[i]
  E = y_test[i]
  _, _,_, A2 = forward(X,www1, bb1, www2, bb2)
  if A2.index(max(A2)) == E.index(max(E)):
    number +=1
print(number/len(y_test))


0.8711785714285715


In [89]:
w1_save = ''
for i in range(len(w1)):
    w1_save += f"{w1[i][0]}" 
    for x in range(1,len(w1[i])):
        w1_save += f",{w1[i][x]}"
    w1_save +='\n'
with open('w1.txt','w') as file:
    file.write(w1_save)

w2_save = ''
for i in range(len(w2)):
    w2_save += f"{w2[i][0]}" 
    for x in range(1,len(w2[i])):
        w2_save += f",{w2[i][x]}"
    w2_save +='\n'
with open('w2.txt','w') as file:
    file.write(w2_save)
    
b1_save = ''
b1_save += f"{b1[0]}" 
for x in range(1,len(b1)):
    b1_save += f",{b1[x]}"
with open('b1.txt','w') as file:
    file.write(b1_save)

b2_save = ''
b2_save += f"{b2[0]}" 
for x in range(1,len(b2)):
    b2_save += f",{b2[x]}"
with open('b2.txt','w') as file:
    file.write(b2_save)

In [ ]:
X_test = DAT[42000:]
y_test = RES[42000:]
number = 0
for i in range(len(y_test)):  
  X = X_test[i]
  E = y_test[i]
  _, _,_, A2 = forward(X,www1, bb1, www2, bb2)
  if A2.index(max(A2)) == E.index(max(E)):
    number +=1
print(number/len(y_test))
